

Investigate a Dataset-[noshowappointments-kagglev2-may-2016.csv]
 
## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction
This dataset collects information from 100k medical appointments in Brazil and is focused on the question of whether or not patients show up for their appointment. A number of characteristics about the patient are included in each row.

‘ScheduledDay’ tells us on what day the patient set up their appointment.
‘Neighborhood’ indicates the location of the hospital.
‘Scholarship’ indicates whether or not the patient is enrolled in Brasilian welfare program Bolsa Família.
Be careful about the encoding of the last column: it says ‘No’ if the patient showed up to their appointment, and ‘Yes’ if they did not show up
### Dataset Description 
we have CSV file which contain the data we are going to analyze


### Question(s) for Analysis
what factors are important for us to know in order to predict if pationt will show up for there scheduled appointment

In [ ]:
# Use this cell to set up import statements for all of the packages that you
#   plan to use.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snb
%matplotlib inline




In [ ]:
# Upgrade pandas to use dataframe.explode() function. 
!pip install --upgrade pandas==0.25.0

<a id='wrangling'></a>
## Data Wrangling

in this section we will load the data , check for cleanliness and then trim and clean the dataset for analysis







In [ ]:
General properties
#loading data and print out a few lines perform operations to inspect data
# types and look for instances of missing or pcssibly errant data
df = pd.read_csv('noshowappointments-kagglev2-may-2016.csv)
df.head()                 

In [ ]:
#exploring the shape of data
df.shape

In [ ]:
#check for duplication
df.duplicate-ed.sum()

In [ ]:
#chech for number of unique value

In [ ]:
#check the number of duplicated patients IDS

<a id='eda'></a>
## Exploratory Data Analysis






In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

 

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])